**тематическое моделирование**

Евгений Борисов borisov.e@solarl.ru

In [1]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

# разложение частотной матрицы [ слова x документы ]  
#
# получаем матрицу с описанием тем [ слова х темы ]   
# и матрицу вероятностей событий "тема описывает документ"  [ темы х документы ]
# 
# [ слова x документы ] = [ слова х темы ] * [ темы х документы ]
# 
# p(w|d) = p(w|t) * p(t|d)

In [2]:
import sys
import re
import gzip
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [4]:
pd.options.display.max_colwidth = 200  

In [5]:
n_features = 1000
n_components = 10

df = pd.read_pickle('../data/text/news.pkl.gz')
print('текстов:',len(df))
df.sample(4)

текстов: 3196


,text,tag
1665,"МОСКВА, 13 дек — РИА Новости. Избранный президент США Дональд Трамп стал ""человеком года"" по версии газеты Financial Times.\n\n""Он переписал правила американской политики. Теперь президент-организ...",politics
1067,"Зарплаты белорусских сенаторов в 2,6 раза больше, чем в среднем по стране\n\n28 ноября 2016 в 11:47\n\nFINANCE.TUT.BY\n\nЕсли исходить из информации, озвученной членом Совета Республики, лидером Л...",economics
2176,Вратарь «Филадельфии» Энтони Столарц признан первой звездой дня в НХЛ. Игрок провел сухой матч против «Детройта» (1:0) в регулярном чемпионате НХЛ. Это девятая победа команды кряду. Столарц отрази...,sport
2200,На концерте музыканта Стаса Барецкого и участника реалити-шоу Николая Должанского девушка брызнула им в лица из перцового баллончика.\n\n11:19,incident


In [6]:
data = df['text'].tolist()

In [7]:
# from nltk import download as nltk_download
# nltk_download('stopwords')

In [8]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

# from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords

stopwords = set(nltk_stopwords.words('russian') )
print('количество стоп-слов:',len(stopwords))

sorted(stopwords)

количество стоп-слов: 151


['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [9]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

CPU times: user 1.75 s, sys: 125 ms, total: 1.87 s
Wall time: 1.9 s


In [10]:
df[['text_clean']].sample(10)

,text_clean
2063,"[москва, декабря, тасс, игрок, киевского, динамо, андрей, ярмоленко, признан, лучшим, футболистом, украины, году, сообщается, официальном, сайте, федерации, футбола, украины, награду, также, прете..."
245,"[виктор, ющенко, намерен, снять, фамилию, избирательного, списка, нсну, президент, украины, виктор, ющенко, намерен, снять, фамилию, избирательного, списка, нсну, допустить, раскола, команде, свои..."
2521,"[плану, элона, маска, доставке, человеческого, экипажа, космос, помощью, капсулы, dragon, году, суждено, сбыться, хотя, spacex, планировала, запуск, пилотируемого, полета, помощью, nasa, следующем..."
403,"[ожирение, подростков, ведет, заболеваниям, печени, специалисты, британского, национального, форума, проблемам, ожирения, отмечают, значительное, увеличение, частоты, случаев, цирроза, печени, осо..."
1793,"[коллегия, счетной, палаты, российской, федерации, председательством, татьяны, голиковой, рассмотрела, форме, видеоконференции, субъектами, российской, федерации, результаты, экспертно, аналитичес..."
116,"[днепр, достойно, простился, кубком, уефа, днепр, который, неделю, назад, потерял, шансы, выход, следующий, круг, напоследок, хлопнул, дверью, очень, громко, обыграв, швейцарии, грасхопперс, счето..."
157,"[гормон, стресса, мешает, работе, иммунной, системы, установили, гормон, названием, нейропептид, вырабатываемый, нейронами, головного, мозга, стрессовых, ситуациях, препятствует, нормальной, работ..."
3170,"[организации, беломорские, новости, belomornews, северодвинск, оленеводы, мира, придут, помощь, ненецким, оленеводам, ненецкий, автономный, округ, главных, оленеводческих, регионов, россии, мира, ..."
2391,"[танец, юной, жительницы, ставрополя, покорил, зрителей, всей, россии, восьмилетняя, божена, дроваль, борется, победу, всероссийском, конкурсе, проекта, канала, россия, синяя, птица, танец, девочк..."
508,"[сложились, карточный, домик, бобруйске, рухнули, гаража, строящемся, кооперативе, декабря, дмитрий, суслов, коммерческий, курьер, четыре, гаража, обрушились, декабря, бобруйске, случилось, улице,..."


In [11]:
# sorted(set([ w for t in  df['text_clean'] for w in t ]))

In [12]:
# собираем слова в строку
df['text_clean'] = df['text_clean'].apply(lambda t: ' '.join(t) )

In [13]:
%xdel stopwords

In [14]:
df[ df['text_clean'].str.len()<1 ]

,text,tag,text_clean


In [15]:
df.sample(10)

,text,tag,text_clean
1022,"Белорусскую ""белочку"" продают за 750 долларов. На аукцион выставлена редкая банкнота в 50 копеек\n\n9 декабря 2016 в 11:17\n\nFINANCE.TUT.BY\n\nНа белорусском аукционе появился редкий и дорогой ло...",economics,белорусскую белочку продают долларов аукцион выставлена редкая банкнота копеек декабря finance tut белорусском аукционе появился редкий дорогой лот банкнота образец копеек года сообщает сайт белор...
2494,"В польском Вроцлаве в 29-й раз вручили награды Европейской киноакадемии. Лучшим европейским фильмом года признана трагикомедия Марен Аде ""Тони Эрдманн"". Россия тоже получила приз - лучшим композит...",culture,польском вроцлаве вручили награды европейской киноакадемии лучшим европейским фильмом года признана трагикомедия марен аде тони эрдманн россия получила приз лучшим композитором назван илья демуцки...
2423,"Трагикомедия Демьяна Шазела ""Ла-Ла Ленд"" признана лучшим фильмом этого года. Critic`s Choice присуждается ежегодно и считается своеобразной подготовкой к церемонии вручения премии американской кин...",culture,трагикомедия демьяна шазела ленд признана лучшим фильмом года critic choice присуждается ежегодно считается своеобразной подготовкой церемонии вручения премии американской киноакадемии оскар премь...
517,"Граната рядом с трассой. Черный копатель ""заминировал"" дорогу под Витебском\n\n8 декабря 2016 в 8:11\n\nЕлена Бегунова, СБ Беларусь Сегодня\n\nПроезжая недалеко от городского поселка Руба, водител...",incident,граната рядом трассой черный копатель заминировал дорогу витебском декабря елена бегунова беларусь сегодня проезжая недалеко городского поселка руба водитель международник александр ратькович глуб...
821,"Обезьянка Пастернака, любовь на полвека и козни спецслужб. Классик беллита без глянца\n\n2 декабря 2016 в 12:18\n\nДенис Мартинович / TUT.BY\n\nАнна Северинец, которая преподает русскую литературу...",social,обезьянка пастернака любовь полвека козни спецслужб классик беллита глянца декабря денис мартинович tut анна северинец которая преподает русскую литературу одной школ смолевичей составила популярн...
908,"Около 85% жителей Сельхозпоселка считают, что в Котовке не должны ничего строить\n\n1 декабря 2016 в 13:05\n\nЕлена Толкачева / REALTY.TUT.BY\n\nОпрос среди жителей Сельхозпоселка в течение почти ...",realty,около жителей сельхозпоселка считают котовке должны строить декабря елена толкачева realty tut опрос среди жителей сельхозпоселка течение трех недель ноября проводило товарищество зеленая сеть цен...
153,"США помогут Китаю минимизировать ущерб от отравления реки Сунгари Как\nуже сообщало ИА REGNUM , Пекин ранее извинился за загрязнение Сунгари\nнитробензолом в результате аварии на химическом заводе...",health,сша помогут китаю минимизировать ущерб отравления реки сунгари сообщало regnum пекин ранее извинился загрязнение сунгари нитробензолом результате аварии химическом заводе китае пятно движется стор...
557,"В Могилеве на Днепре пришлось спасать троих любителей зимней рыбалки\n\n3 декабря 2016 в 18:42\n\nTUT.BY\n\nДнем 2 декабря помощь понадобилась троим любителям зимней рыбалки. Мужчины, которые вышл...",incident,могилеве днепре пришлось спасать троих любителей зимней рыбалки декабря tut днем декабря помощь понадобилась троим любителям зимней рыбалки мужчины которые вышли лед залива реки днепр районе улицы...
490,Перестрелка по недоразумению между силовиками под Киевом: уже шесть погибших\n\nопубликовано: 4 декабря 2016 в 12:30\n\nобновлено: 4 декабря 2016 в 21:29\n\nУкраинская правда\n\nВ ночь на воскресе...,incident,перестрелка недоразумению силовиками киевом шесть погибших опубликовано декабря обновлено декабря украинская правда ночь воскресенье поселке княжичи броварского района результате перестрелки сотру...
768,"В Бресте открылась предновогодняя ярмарка\n\n7 декабря 2016 в 8:09\n\nОксана Козляковская, Вечерний Брест\n\n6 декабря на улице Гоголя заработала предновогодняя ярмарка, пишет «Вечерний Брест».\n\...",social,бресте открылась предновогодн

---

In [16]:
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        message = "Тема %d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

---

In [17]:
#df['text_clean']

In [18]:
# tf features 
tf_vectorizer = CountVectorizer( max_df=0.95, min_df=2, max_features=n_features )
tf = tf_vectorizer.fit_transform(df['text_clean'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [19]:
# LDA - латентное размещение Дирихле
lda = LatentDirichletAllocation( n_components=n_components, max_iter=5, 
                                learning_method='online', learning_offset=50.,
                                random_state=0 ).fit(tf)
print('\nLDA:\n')
print_top_words(lda, tf_feature_names)


LDA:

Тема 0: это очень которые время просто лет нужно
Тема 1: президент сша трамп президента заявил декабря глава
Тема 2: рублей автомобиль модели автомобилей модель автомобиля авто
Тема 3: беларуси tut декабря установлена внимание версия браузер
Тема 4: народов коренных севера малочисленных коми проекта края
Тема 5: object савченко нефти нефть тгк индекс оао
Тема 6: года году это также компании декабря которые
Тема 7: декабря tut время дтп фото результате области
Тема 8: года также декабря россии тысяч сообщает данным
Тема 9: динамо лучший мяч мира место стал чемпионата


---

In [20]:
# tf-idf features 
tfidf_vectorizer = TfidfVectorizer( max_df=0.95, min_df=2, max_features=n_features)

tfidf = tfidf_vectorizer.fit_transform(df['text_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [21]:
# NMF (Frobenius norm) - неотрицательное матричное разложение
nmf = NMF( n_components=n_components, random_state=1,alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(Frobenius norm):\n')
print_top_words( nmf, tfidf_feature_names )


NMF(Frobenius norm):

Тема 0: это года которые году также декабря время
Тема 1: трамп сша трампа дональд президент избранный президента
Тема 2: дтп водитель результате области мвд декабря происшествия
Тема 3: flash adobe javascript player проигрывателя html5 браузер
Тема 4: савченко украины партии надежда заявила лидер действия
Тема 5: рублей млн млрд долларов тысяч года году
Тема 6: динамо чемпионата матче мира очков матча лиги
Тема 7: народов севера коренных малочисленных края фестиваль июля
Тема 8: россии путин президент президента заявил глава россия
Тема 9: алеппо города сирии жителей восточной сутки тысяч


---

In [22]:
# NMF (generalized Kullback-Leibler divergence)  
nmf = NMF( n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
          solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(generalized Kullback-Leibler divergence):\n')
print_top_words(nmf, tfidf_feature_names )


NMF(generalized Kullback-Leibler divergence):

Тема 0: декабря года это которая также году фото
Тема 1: сша президент года ранее президента главы дональд
Тема 2: декабря области время результате tut пресс сообщает
Тема 3: tut декабря adobe версия видео браузер внимание
Тема 4: сегодня украины вопрос это декабря словам сообщил
Тема 5: млн будут года области рублей рамках народов
Тема 6: место мира декабря который стал матче года
Тема 7: ученые жизни которые лет людей люди могут
Тема 8: россии декабря заявил глава государства президент россия
Тема 9: года также компания который сообщает ранее компании


In [ ]:
удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])